In [ ]:
import pandas as pd
def read_file(filepath):
    file=pd.read_csv(filepath)
    return file
    # lines=zip(file.HADM_ID,file.Label)
    # return lines

filepath = "/PatientTM/data/extended/discharge/train.csv"
df_train = read_file(filepath)
df_train = df_train[df_train.Label == 0].HADM_ID
negative_HADMIDs_train = [hadm_id for hadm_id in df_train]

filepath = "/PatientTM/data/extended/discharge/val.csv"
df_val = read_file(filepath)
df_val = df_val[df_val.Label == 0].HADM_ID
negative_HADMIDs_val = [hadm_id for hadm_id in df_val]
                        
filepath = "/PatientTM/data/extended/discharge/test.csv"
df_test = read_file(filepath)
df_test = df_test[df_test.Label == 0].HADM_ID
negative_HADMIDs_test = [hadm_id for hadm_id in df_test]

negative_HADMIDs = negative_HADMIDs_train + negative_HADMIDs_val + negative_HADMIDs_test
with open("/PatientTM/data/extended/discharge/negative_hadmids.txt", "w") as file:
    for hadm in negative_HADMIDs:
        file.write(str(hadm)+"\n")
       

In [2]:
import pandas as pd
import numpy as np

with open("/PatientTM/data/extended/discharge/negative_hadmids.txt","r") as file:
    lines = file.readlines()
    negative_HADMIDs = [line.strip("\n") for line in lines]
    
def read_precomputed_npy(input_file):
    """Reads a comma separated value file."""
    array = np.load(input_file, allow_pickle=True)
    return array

def df_to_dict_array(pandas_df):
    entries = []
    for index, row in pandas_df.iterrows():
        entry = {}
        entry.update({"HADM_ID": row["HADM_ID"], "clinicalbert_embedding": row["clinicalbert_embedding"]})
        entries.append(entry)
    return(entries)
    
    
# df_train = pd.read_csv("/PatientTM/data/extended/discharge/train.csv")
# df_val   = pd.read_csv("/PatientTM/data/extended/discharge/val.csv")
# df_test  = pd.read_csv("/PatientTM/data/extended/discharge/test.csv")
# df_total = pd.concat([df_train, df_val, df_test])

df_total = pd.read_csv("/PatientTM/data/extended_folds/discharge/fold0_text.csv")
array = read_precomputed_npy("/PatientTM/data/extended_folds/discharge/fold0_text_precomputed.npy")
for i in range(1,10):
    df_fold = pd.read_csv("/PatientTM/data/extended_folds/discharge/fold"+str(i)+"_text.csv")
    df_total = pd.concat([df_total, df_fold])
    array_fold = read_precomputed_npy("/PatientTM/data/extended_folds/discharge/fold" + str(i) + "_text_precomputed.npy")
    array = np.concatenate([array, array_fold])
                                 
precomputed_df = pd.DataFrame.from_dict(list(array))
                                 
positive_HADMIDs = df_total[df_total.Label == 1.0].HADM_ID
positive_HADMIDs = list(np.unique(positive_HADMIDs))
negative_HADMIDs = list(np.unique(negative_HADMIDs))
valid_HADMIDs = negative_HADMIDs + positive_HADMIDs

df_total_valid = df_total[df_total.HADM_ID.isin(valid_HADMIDs)]
df_total_valid_HADMID = df_total_valid.groupby(['HADM_ID'])['HADM_ID'].nth(0).reset_index(drop=True)
valid_labels = df_total_valid.groupby(['HADM_ID'])['Label'].agg(np.min).reset_index(drop=True)
# print(valid_labels)
# print(np.unique(df_total_valid.Label, return_counts=True))
    
from sklearn.model_selection import StratifiedKFold
skfold = StratifiedKFold(n_splits = 5, shuffle=True)
i = 0
for train_index, test_index in skfold.split(X=df_total_valid_HADMID, y=valid_labels):
    print("Fold Number {}:\n".format(i))
    fold_HADM_IDs = df_total_valid_HADMID[test_index]
    fold_df = df_total_valid[df_total_valid.HADM_ID.isin(fold_HADM_IDs)].reset_index(drop=True)
    filename = '/PatientTM/data/extended/discharge/fold' + str(i) + '_text.csv'
    fold_df.to_csv(filename, columns=["SUBJECT_ID","HADM_ID","ADMITTIME","DAYS_NEXT_ADMIT","DAYS_PREV_ADMIT","DURATION","DIAG_ICD9","SMALL_DIAG_ICD9","DIAG_CCS","PROC_ICD9","SMALL_PROC_ICD9","PROC_CCS","NDC","CUI","Label","TEXT","NEXT_SMALL_DIAG_ICD9","NEXT_DIAG_CCS","NEXT_SMALL_PROC_ICD9","NEXT_PROC_CCS","NEXT_CUI"])
    fold_precomputed_df = precomputed_df[precomputed_df.HADM_ID.isin(fold_HADM_IDs)].reset_index(drop=True)
    fold_precomputed_entries = df_to_dict_array(fold_precomputed_df)
    fold_array = np.asarray(fold_precomputed_entries, dtype=object) 
    array_filepath = "/PatientTM/data/extended/discharge/fold" + str(i) + "_text_precomputed.npy"
    np.save(array_filepath, fold_array)
    i+=1
    

Fold Number 0:

Fold Number 1:

Fold Number 2:

Fold Number 3:

Fold Number 4:



In [16]:
import pandas as pd
# df_test = pd.read_csv("/PatientTM/data/extended/discharge/test.csv")
# print(np.unique(df_test.Label, return_counts=True)[1])
df_0 = pd.read_csv("/PatientTM/data/extended/discharge/fold0_text.csv")
df_1 = pd.read_csv("/PatientTM/data/extended/discharge/fold1_text.csv")
df_2 = pd.read_csv("/PatientTM/data/extended/discharge/fold2_text.csv")
df_3 = pd.read_csv("/PatientTM/data/extended/discharge/fold3_text.csv")
df_4 = pd.read_csv("/PatientTM/data/extended/discharge/fold4_text.csv")
# print(np.unique(df_0.Label, return_counts=True)[1])
# print(np.unique(df_1.Label, return_counts=True)[1])
# print(np.unique(df_2.Label, return_counts=True)[1])
# print(np.unique(df_3.Label, return_counts=True)[1])
# print(np.unique(df_4.Label, return_counts=True)[1])

# df_0_HADM_ID = df_0.groupby(['HADM_ID'])['HADM_ID'].nth(0).reset_index(drop=True)
df_0_HADM_ID = set(df_0.HADM_ID)
df_1_HADM_ID = set(df_1.HADM_ID)
df_2_HADM_ID = set(df_2.HADM_ID)
df_3_HADM_ID = set(df_3.HADM_ID)
df_4_HADM_ID = set(df_4.HADM_ID)

print(f"Intersection between folds 0 / 1: {df_0_HADM_ID.intersection(df_1_HADM_ID)}")
df_global = set.union(df_0_HADM_ID, df_1_HADM_ID)
# print(len(df_0_HADM_ID),len(df_1_HADM_ID), len(df_global))

print(f"Intersection between folds 0+1 / 2: {df_global.intersection(df_2_HADM_ID)}")
df_global = set.union(df_global, df_2_HADM_ID)
# print(len(df_2_HADM_ID), len(df_global))

print(f"Intersection between folds 0+1+2 / 3: {df_global.intersection(df_3_HADM_ID)}")
df_global = set.union(df_global, df_3_HADM_ID)
# print(len(df_3_HADM_ID), len(df_global))

print(f"Intersection between folds 0+1+2+3 / 4: {df_global.intersection(df_4_HADM_ID)}")
df_global = set.union(df_global, df_4_HADM_ID)
# print(len(df_4_HADM_ID), len(df_global))




Intersection between folds 0 / 1: set()
Intersection between folds 0+1 / 2: set()
Intersection between folds 0+1+2 / 3: set()
Intersection between folds 0+1+2+3 / 4: set()


In [37]:
import pandas as pd
df_0 = pd.read_csv("/PatientTM/data/extended/discharge/fold0_text.csv")
df_1 = pd.read_csv("/PatientTM/data/extended/discharge/fold1_text.csv")
df_2 = pd.read_csv("/PatientTM/data/extended/discharge/fold2_text.csv")
df_3 = pd.read_csv("/PatientTM/data/extended/discharge/fold3_text.csv")
df_4 = pd.read_csv("/PatientTM/data/extended/discharge/fold4_text.csv")

df_0_notext = df_0.drop(["TEXT"], axis=1)
df_0_notext = df_0_notext.drop_duplicates(subset=["HADM_ID"])
df_0_notext.to_csv("/PatientTM/data/extended/discharge/fold0_notext.csv", columns=["SUBJECT_ID","HADM_ID","ADMITTIME","DAYS_NEXT_ADMIT","DAYS_PREV_ADMIT","DURATION","DIAG_ICD9","SMALL_DIAG_ICD9","DIAG_CCS","PROC_ICD9","SMALL_PROC_ICD9","PROC_CCS","NDC","CUI","Label","NEXT_SMALL_DIAG_ICD9","NEXT_DIAG_CCS","NEXT_SMALL_PROC_ICD9","NEXT_PROC_CCS","NEXT_CUI"])

df_1_notext = df_1.drop(["TEXT"], axis=1)
df_1_notext = df_1_notext.drop_duplicates(subset=["HADM_ID"])
df_1_notext.to_csv("/PatientTM/data/extended/discharge/fold1_notext.csv", columns=["SUBJECT_ID","HADM_ID","ADMITTIME","DAYS_NEXT_ADMIT","DAYS_PREV_ADMIT","DURATION","DIAG_ICD9","SMALL_DIAG_ICD9","DIAG_CCS","PROC_ICD9","SMALL_PROC_ICD9","PROC_CCS","NDC","CUI","Label","NEXT_SMALL_DIAG_ICD9","NEXT_DIAG_CCS","NEXT_SMALL_PROC_ICD9","NEXT_PROC_CCS","NEXT_CUI"])

df_2_notext = df_2.drop(["TEXT"], axis=1)
df_2_notext = df_2_notext.drop_duplicates(subset=["HADM_ID"])
df_2_notext.to_csv("/PatientTM/data/extended/discharge/fold2_notext.csv", columns=["SUBJECT_ID","HADM_ID","ADMITTIME","DAYS_NEXT_ADMIT","DAYS_PREV_ADMIT","DURATION","DIAG_ICD9","SMALL_DIAG_ICD9","DIAG_CCS","PROC_ICD9","SMALL_PROC_ICD9","PROC_CCS","NDC","CUI","Label","NEXT_SMALL_DIAG_ICD9","NEXT_DIAG_CCS","NEXT_SMALL_PROC_ICD9","NEXT_PROC_CCS","NEXT_CUI"])

df_3_notext = df_3.drop(["TEXT"], axis=1)
df_3_notext = df_3_notext.drop_duplicates(subset=["HADM_ID"])
df_3_notext.to_csv("/PatientTM/data/extended/discharge/fold3_notext.csv", columns=["SUBJECT_ID","HADM_ID","ADMITTIME","DAYS_NEXT_ADMIT","DAYS_PREV_ADMIT","DURATION","DIAG_ICD9","SMALL_DIAG_ICD9","DIAG_CCS","PROC_ICD9","SMALL_PROC_ICD9","PROC_CCS","NDC","CUI","Label","NEXT_SMALL_DIAG_ICD9","NEXT_DIAG_CCS","NEXT_SMALL_PROC_ICD9","NEXT_PROC_CCS","NEXT_CUI"])

df_4_notext = df_4.drop(["TEXT"], axis=1)
df_4_notext = df_4_notext.drop_duplicates(subset=["HADM_ID"])
df_4_notext.to_csv("/PatientTM/data/extended/discharge/fold4_notext.csv", columns=["SUBJECT_ID","HADM_ID","ADMITTIME","DAYS_NEXT_ADMIT","DAYS_PREV_ADMIT","DURATION","DIAG_ICD9","SMALL_DIAG_ICD9","DIAG_CCS","PROC_ICD9","SMALL_PROC_ICD9","PROC_CCS","NDC","CUI","Label","NEXT_SMALL_DIAG_ICD9","NEXT_DIAG_CCS","NEXT_SMALL_PROC_ICD9","NEXT_PROC_CCS","NEXT_CUI"])

